In [1]:
import pandas as pd

# データフレームの作成
data = {
    'Points limit': [500, 1000, 10000, 500, 1000, 10000],
    'Proposed (5%)': [1, 9, 0.083, 25, 33, 0.106],
    'ORB': [2, 10, 0.361, 26, 34, 0.576],
    # 他の列も同様に追加
}
df = pd.DataFrame(data)

# LaTeX形式で基本的なテーブルを出力
latex_table = df.to_latex(index=False)
print(latex_table)

\begin{tabular}{rrr}
\toprule
 Points limit &  Proposed (5\%) &    ORB \\
\midrule
          500 &          1.000 &  2.000 \\
         1000 &          9.000 & 10.000 \\
        10000 &          0.083 &  0.361 \\
          500 &         25.000 & 26.000 \\
         1000 &         33.000 & 34.000 \\
        10000 &          0.106 &  0.576 \\
\bottomrule
\end{tabular}



In [2]:
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
import math
from decimal import Decimal

COMPARE_DESCRIPTORS = ["Proposed1", "Proposed",  "orb", "sphorb", "sift", "spoint", "alike", ]
ABLATION_DESCRIPTORS = ["Proposed1", "Proposed", "Proposed01", "Proposed10", "Ltspoint", "Proposed_un", "tspoint", "Ftspoint"] 
NOT_T_DESCRIPTORS = ["spoint", "tspoint", "sphorb", "Ltspoint", "Proposed01", "Proposed03", "Proposed05", "Proposed1", "Proposed3", "Proposed", "Proposed10", "Proposed20", "Proposed_un", "Ftspoint"]
METHODS = ["", "t"]
PARAMS = ["R", "T"]
PARAMS_DICT = {"R": "Rotation", "T": "Translation"}
ALL_LOCS = ["Classroom", "Room", "Realistic", "Interior1", "Interior2", "Urban1", "Urban2", "Urban3", "Urban4"]
INDOORS = ["Classroom", "Room", "Realistic", "Interior1", "Interior2"]
OUTDOORS = ["Urban1", "Urban2", "Urban3", "Urban4"]
POSES = ["pose1", "pose2", "pose3", "pose4", "pose5"]
TIME_CATEGORYS = ["FP", "MC", "PE"]
DESCRIPTORS_DICT = {"Proposed1": "Proposed (1%)",
                    "Proposed01": "Proposed (0.1%)",
                    "Proposed03": "Proposed (0.3%)",
                    "Proposed05": "Proposed (0.5%)",
                    "Proposed3": "Proposed (3%)",
                    "Proposed": "Proposed (5%)",
                    "Proposed10": "Proposed (10%)",
                    "Proposed20": "Proposed (20%)",
                    "Proposed_un": "MNN",
                    "orb": "ORB", 
                    "sphorb": "SPHORB",
                    "sift": "SIFT",
                    "alike": "ALIKE",
                    "spoint": "SPoint",
                    "tspoint": "KNN+L",
                    "Ltspoint": "Proposed (5%)+L",
                    "Ftspoint": "FLANN"
                    }
METHODS_DICT = {"t": "T", "":""}
INDEX_CATEGORYS_PATH = ["MATCHING_NUM", "MATCHING_SCORE", "MEAN_MATCHING_ACCURCY"]
INDEX_CATEGORYS = ["MN", ]

In [3]:
def read_csv_data(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        data = [float(row[0]) for row in reader]
    return data

In [4]:
def make_latex_table_time():
    for i, param in enumerate(PARAMS):
        for j, loc in enumerate(["indoor", "outdoor"]):
            if loc == "indoor":
                locs = INDOORS
            else:
                locs = OUTDOORS
            time_data = {'Points limit': ["ALL", 500, 1000, 10000]}
            time_compare_df = pd.DataFrame(time_data)
            time_ablation_df = pd.DataFrame(time_data)
            for tgt in ["compare", "ablation"]:
                if tgt == "compare":
                    descriptors = COMPARE_DESCRIPTORS
                    time_df = time_compare_df
                else:
                    descriptors = ABLATION_DESCRIPTORS
                    time_df = time_ablation_df
                for descriptor in descriptors:
                    for method in METHODS:
                        if descriptor in NOT_T_DESCRIPTORS and method == "t":
                            continue
                        method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                        time_df[method_tmp] = [0, 0, 0, 0]
        
            for idx_points, num_points in enumerate([500, 1000, 10000]):
                base_path = f"results/FP_{num_points}/values"
                value_dict = {}
                for tgt in ["compare", "ablation"]:
                    if tgt == "compare":
                        descriptors = COMPARE_DESCRIPTORS
                        time_df = time_compare_df
                    else:
                        descriptors = ABLATION_DESCRIPTORS
                        time_df = time_ablation_df
                    for time_category in TIME_CATEGORYS:
                        #print(time_category)
                        for descriptor in descriptors:
                            for method in METHODS:
                                if descriptor in NOT_T_DESCRIPTORS and method == "t":
                                    continue
                                method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                                all_time_data = []
                                for scene in ALL_LOCS:
                                    file_path = f"{base_path}/{scene}_{method}{descriptor}_5PA_GSM_wRT/TIMES_{time_category}.csv"
                                    time_data = read_csv_data(file_path)
                                    all_time_data.extend(time_data)
                                if time_category == "FP":
                                    time_df.loc[0, method_tmp] = np.mean(all_time_data)
                                elif time_category == "MC":
                                    time_df.loc[idx_points+1, method_tmp] = np.mean(all_time_data)
        
            time_latex_table = time_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
            print(time_latex_table)
            time_latex_table = time_ablation_df.to_latex(index=False, float_format="{:.3f}".format)
            print(time_latex_table)

make_latex_table_time()

\begin{tabular}{lrrrrrrrrrr}
\toprule
Points limit &  Proposed (1\%) &  Proposed (5\%) &   ORB &  TORB &  SPHORB &  SIFT &  TSIFT &  SPoint &  ALIKE &  TALIKE \\
\midrule
         ALL &          0.504 &          0.486 & 0.195 & 0.222 &   0.276 & 0.228 &  0.662 &   0.240 &  0.189 &   0.334 \\
         500 &          0.013 &          0.008 & 0.010 & 0.013 &   0.001 & 0.010 &  0.011 &   0.011 &  0.010 &   0.011 \\
        1000 &          0.018 &          0.013 & 0.010 & 0.013 &   0.002 & 0.010 &  0.011 &   0.011 &  0.010 &   0.011 \\
       10000 &          0.021 &          0.015 & 0.082 & 0.057 &   0.103 & 0.011 &  0.011 &   0.013 &  0.012 &   0.012 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrrrr}
\toprule
Points limit &  Proposed (1\%) &  Proposed (5\%) &  Proposed (0.1\%) &  Proposed (10\%) &  Proposed (5\%)+L &   MNN &  KNN+L &  FLANN \\
\midrule
         ALL &          0.504 &          0.486 &            0.498 &           0.499 &            0.486 & 0.485 &  0.487 &  0.485 \\


In [12]:
def make_latex_table_INDEX():
    for j, loc in enumerate(["indoor", "outdoor"]):
        if loc == "indoor":
            locs = INDOORS
        else:
            locs = OUTDOORS
                        
        for idx_points, num_points in enumerate([500, 1000, 10000]):
            base_path = f"results/FP_{num_points}/values"
            print(loc, num_points)
            for tgt in ["compare", "ablation"]:
                if tgt == "compare":
                    descriptors = COMPARE_DESCRIPTORS
                    method_tmp_list = []
                    for descriptor in descriptors:
                        for method in METHODS:
                            if descriptor in NOT_T_DESCRIPTORS and method == "t":
                                continue
                            method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                            method_tmp_list.append(method_tmp)
                    maemse_data = {'Method': method_tmp_list}
                    maemse_compare_df = pd.DataFrame(maemse_data)
                    maemse_df = maemse_compare_df
                else:
                    descriptors = ABLATION_DESCRIPTORS
                    method_tmp_list = []
                    for descriptor in descriptors:
                        for method in METHODS:
                            if descriptor in NOT_T_DESCRIPTORS and method == "t":
                                continue
                            method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                            method_tmp_list.append(method_tmp)
                    maemse_data = {'Method': method_tmp_list}
                    maemse_ablation_df = pd.DataFrame(maemse_data)
                    maemse_df = maemse_ablation_df


                for method_tmp in method_tmp_list:
                    for j in range(8):
                       maemse_df[j] = [0] * len(method_tmp_list)


                idx_methods = 0
                for descriptor in descriptors:
                    for method in METHODS:
                        if descriptor in NOT_T_DESCRIPTORS and method == "t":
                            continue
                        all_FN_data = []
                        all_MN_data = []
                        all_MS_data = []
                        all_MMA_data = []
                        all_errorR_data = []
                        all_errorT_data = []
                        method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"

                        for scene in locs:
                            file_base_path = f"{base_path}/{scene}_{method}{descriptor}_5PA_GSM_wRT"
                            FN_path = file_base_path + "/FP_NUM.csv"
                            FN_data = read_csv_data(FN_path)
                            all_FN_data.extend(FN_data)
                            MN_path = file_base_path + "/MATCHING_NUM.csv"
                            MN_data = read_csv_data(MN_path)
                            all_MN_data.extend(MN_data)
                            MS_path = file_base_path + "/MATCHING_SCORE.csv"
                            MS_data = read_csv_data(MS_path)
                            all_MS_data.extend(MS_data)
                            MMA_path = file_base_path + "/MEAN_MATCHING_ACCURCY.csv"
                            MMA_data = read_csv_data(MMA_path)
                            all_MMA_data.extend(MMA_data)
                            errorR_path = file_base_path + f"/R_ERRORS.csv"
                            errorR_data = read_csv_data(errorR_path)
                            all_errorR_data.extend(errorR_data)
                            errorT_path = file_base_path + f"/T_ERRORS.csv"
                            errorT_data = read_csv_data(errorT_path)
                            all_errorT_data.extend(errorT_data)
                        
                        maemse_df.loc[idx_methods, 0] = format(Decimal(str(np.mean(all_FN_data))), '.3g')
                        maemse_df.loc[idx_methods, 1] = format(Decimal(str(np.mean(all_MN_data))), '.3g')
                        maemse_df.loc[idx_methods, 2] = format(Decimal(str(np.mean(all_MS_data))), '.3g')
                        maemse_df.loc[idx_methods, 3] = format(Decimal(str(np.mean(all_MMA_data))), '.3g')
                        maemse_df.loc[idx_methods, 4] = format(Decimal(str(np.mean(all_errorR_data))), '.3g')
                        maemse_df.loc[idx_methods, 5] = format(Decimal(str(np.mean(np.array(all_errorR_data) ** 2))), '.3g')
                        maemse_df.loc[idx_methods, 6] = format(Decimal(str(np.mean(all_errorT_data))), '.3g')
                        maemse_df.loc[idx_methods, 7] = format(Decimal(str(np.mean(np.array(all_errorT_data) ** 2))), '.3g')
                        idx_methods += 1
        
            maemse_latex_table = maemse_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
            print(maemse_latex_table)
            maemse_latex_table = maemse_ablation_df.to_latex(index=False, float_format="{:.3f}".format)
            print(maemse_latex_table)

make_latex_table_INDEX()

indoor 500
\begin{tabular}{lllllllll}
\toprule
       Method &   0 &    1 &      2 &     3 &      4 &      5 &     6 &      7 \\
\midrule
Proposed (1\%) & 496 &  117 &  0.235 & 0.643 & 0.0877 & 0.0901 & 0.109 & 0.0817 \\
Proposed (5\%) & 496 &  119 &  0.240 & 0.623 & 0.0832 & 0.0779 & 0.112 &  0.131 \\
          ORB & 500 & 72.6 &  0.145 & 0.473 &  0.914 &   1.86 & 0.931 &   1.40 \\
         TORB & 500 & 84.8 &  0.170 & 0.518 &  0.667 &   1.22 & 0.731 &   1.07 \\
       SPHORB & 509 & 83.4 &  0.164 & 0.515 &  0.786 &   1.68 & 0.707 &  0.947 \\
         SIFT & 500 & 33.6 & 0.0672 & 0.712 &  0.544 &  0.932 & 0.663 &   1.03 \\
        TSIFT & 499 & 52.7 &  0.106 & 0.728 &  0.411 &  0.868 & 0.413 &  0.560 \\
       SPoint & 499 & 26.3 & 0.0528 & 0.891 &  0.758 &   2.18 & 0.843 &   2.34 \\
        ALIKE & 500 & 78.9 &  0.158 & 0.547 &   2.97 &   9.30 &  2.95 &   9.21 \\
       TALIKE & 481 & 70.9 &  0.151 & 0.576 &   2.78 &   8.65 &  2.80 &   8.67 \\
\bottomrule
\end{tabular}

\begin{tabula

In [5]:
def make_latex_table_INDEX_old():
    for i, param in enumerate(PARAMS):
        for j, loc in enumerate(["indoor", "outdoor"]):
            if loc == "indoor":
                locs = INDOORS
            else:
                locs = OUTDOORS
            print(param, loc)
            maemse_data = {'Points limit': [500, 1000, 10000, 500, 1000, 10000]}
            maemse_compare_df = pd.DataFrame(maemse_data)
            maemse_ablation_df = pd.DataFrame(maemse_data)
            for tgt in ["compare", "ablation"]:
                if tgt == "compare":
                    descriptors = COMPARE_DESCRIPTORS
                    method_tmp_list = []
                    for descriptor in descriptors:
                        for method in METHODS:
                            if descriptor in NOT_T_DESCRIPTORS and method == "t":
                                continue
                            #method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                            method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                            method_tmp_list.append(method_tmp)
                    maemse_data = {'Method': method_tmp_list}
                    maemse_compare_df = pd.DataFrame(maemse_data)
                    maemse_df = maemse_compare_df
                else:
                    descriptors = ABLATION_DESCRIPTORS
                    method_tmp_list = []
                    for descriptor in descriptors:
                        for method in METHODS:
                            if descriptor in NOT_T_DESCRIPTORS and method == "t":
                                continue
                            method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                            method_tmp_list.append(method_tmp)
                    maemse_data = {'Method': method_tmp_list}
                    maemse_ablation_df = pd.DataFrame(maemse_data)
                    maemse_df = maemse_ablation_df


                for method_tmp in method_tmp_list:
                    for j in range(12):
                       maemse_df[j] = [0] * len(method_tmp_list)

                for idx_points, num_points in enumerate([500, 1000, 10000]):
                    base_path = f"results/FP_{num_points}/values"
                    idx_methods = 0
                    for descriptor in descriptors:
                        for method in METHODS:
                            if descriptor in NOT_T_DESCRIPTORS and method == "t":
                                continue
                            all_MN_data = []
                            all_MS_data = []
                            all_error_data = []
                            method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                            for scene in locs:
                                file_base_path = f"{base_path}/{scene}_{method}{descriptor}_5PA_GSM_wRT"
                                MN_path = file_base_path + "/MATCHING_NUM.csv"
                                MN_data = read_csv_data(MN_path)
                                all_MN_data.extend(MN_data)
                                MS_path = file_base_path + "/MATCHING_SCORE.csv"
                                MS_data = read_csv_data(MS_path)
                                all_MS_data.extend(MS_data)
                                error_path = file_base_path + f"/{param}_ERRORS.csv"
                                error_data = read_csv_data(error_path)
                                all_error_data.extend(error_data)
                            
                            maemse_df.loc[idx_methods, idx_points*4] = format(Decimal(str(np.mean(all_MN_data))), '.3g')
                            maemse_df.loc[idx_methods, idx_points*4+1] = format(Decimal(str(np.mean(all_MS_data))), '.3g')
                            maemse_df.loc[idx_methods, idx_points * 4+2] = format(Decimal(str(np.mean(all_error_data))), '.3g')
                            maemse_df.loc[idx_methods, idx_points * 4+3] = format(Decimal(str(np.mean(np.array(all_error_data) ** 2))), '.3g')
                            idx_methods += 1
        
            maemse_latex_table = maemse_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
            print(maemse_latex_table)
            maemse_latex_table = maemse_ablation_df.to_latex(index=False, float_format="{:.3f}".format)
            print(maemse_latex_table)

make_latex_table_INDEX_old()

R indoor
\begin{tabular}{lllllllllllll}
\toprule
       Method &    0 &      1 &      2 &      3 &    4 &      5 &      6 &     7 &    8 &      9 &     10 &    11 \\
\midrule
Proposed (1\%) &  117 &  0.235 & 0.0877 & 0.0901 &  164 &  0.214 & 0.0863 & 0.106 &  164 &  0.213 & 0.0848 & 0.105 \\
Proposed (5\%) &  119 &  0.240 & 0.0832 & 0.0779 &  165 &  0.215 & 0.0918 & 0.123 &  166 &  0.215 & 0.0835 & 0.106 \\
          ORB & 72.6 &  0.145 &  0.914 &   1.86 &  123 &  0.123 &  0.832 &  1.81 &  562 & 0.0782 &  0.361 & 0.576 \\
         TORB & 84.8 &  0.170 &  0.667 &   1.22 &  153 &  0.153 &  0.559 &  1.14 &  639 &  0.118 &  0.184 & 0.276 \\
       SPHORB & 83.4 &  0.164 &  0.786 &   1.68 &  150 &  0.147 &  0.526 &  1.08 &  882 &  0.105 &  0.246 & 0.460 \\
         SIFT & 33.6 & 0.0672 &  0.544 &  0.932 & 49.5 & 0.0514 &  0.281 & 0.378 &  100 & 0.0513 &  0.129 & 0.105 \\
        TSIFT & 52.7 &  0.106 &  0.411 &  0.868 & 86.1 & 0.0961 &  0.229 & 0.495 &  109 & 0.0985 &  0.125 & 0.214 \\
    

In [14]:
def make_latex_table_INDEX_farm():
        locs = POSES
                        
        for idx_points, num_points in enumerate([500, 1000, 10000]):
            base_path = f"results/FP_{num_points}/values"
            print("farm", num_points)
            for tgt in ["compare", "ablation"]:
                if tgt == "compare":
                    descriptors = COMPARE_DESCRIPTORS
                    method_tmp_list = []
                    for descriptor in descriptors:
                        for method in METHODS:
                            if descriptor in NOT_T_DESCRIPTORS and method == "t":
                                continue
                            method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                            method_tmp_list.append(method_tmp)
                    maemse_data = {'Method': method_tmp_list}
                    maemse_compare_df = pd.DataFrame(maemse_data)
                    maemse_df = maemse_compare_df
                else:
                    descriptors = ABLATION_DESCRIPTORS
                    method_tmp_list = []
                    for descriptor in descriptors:
                        for method in METHODS:
                            if descriptor in NOT_T_DESCRIPTORS and method == "t":
                                continue
                            method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                            method_tmp_list.append(method_tmp)
                    maemse_data = {'Method': method_tmp_list}
                    maemse_ablation_df = pd.DataFrame(maemse_data)
                    maemse_df = maemse_ablation_df


                for method_tmp in method_tmp_list:
                    for j in range(8):
                       maemse_df[j] = [0] * len(method_tmp_list)


                idx_methods = 0
                for descriptor in descriptors:
                    for method in METHODS:
                        if descriptor in NOT_T_DESCRIPTORS and method == "t":
                            continue
                        all_FN_data = []
                        all_MN_data = []
                        all_MS_data = []
                        all_MMA_data = []
                        all_errorR_data = []
                        all_errorT_data = []
                        method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"

                        for scene in locs:
                            file_base_path = f"{base_path}/{scene}_{method}{descriptor}_5PA_GSM_wRT"
                            FN_path = file_base_path + "/FP_NUM.csv"
                            FN_data = read_csv_data(FN_path)
                            all_FN_data.extend(FN_data)
                            MN_path = file_base_path + "/MATCHING_NUM.csv"
                            MN_data = read_csv_data(MN_path)
                            all_MN_data.extend(MN_data)
                            MS_path = file_base_path + "/MATCHING_SCORE.csv"
                            MS_data = read_csv_data(MS_path)
                            all_MS_data.extend(MS_data)
                            MMA_path = file_base_path + "/MEAN_MATCHING_ACCURCY.csv"
                            MMA_data = read_csv_data(MMA_path)
                            all_MMA_data.extend(MMA_data)
                            errorR_path = file_base_path + f"/R_ERRORS.csv"
                            errorR_data = read_csv_data(errorR_path)
                            all_errorR_data.extend(errorR_data)
                            errorT_path = file_base_path + f"/T_ERRORS.csv"
                            errorT_data = read_csv_data(errorT_path)
                            all_errorT_data.extend(errorT_data)
                        
                        maemse_df.loc[idx_methods, 0] = format(Decimal(str(np.mean(all_FN_data))), '.3g')
                        maemse_df.loc[idx_methods, 1] = format(Decimal(str(np.mean(all_MN_data))), '.3g')
                        maemse_df.loc[idx_methods, 2] = format(Decimal(str(np.mean(all_MS_data))), '.3g')
                        maemse_df.loc[idx_methods, 3] = format(Decimal(str(np.mean(all_MMA_data))), '.3g')
                        maemse_df.loc[idx_methods, 4] = format(Decimal(str(np.mean(all_errorR_data))), '.3g')
                        maemse_df.loc[idx_methods, 5] = format(Decimal(str(np.mean(np.array(all_errorR_data) ** 2))), '.3g')
                        maemse_df.loc[idx_methods, 6] = format(Decimal(str(np.mean(all_errorT_data))), '.3g')
                        maemse_df.loc[idx_methods, 7] = format(Decimal(str(np.mean(np.array(all_errorT_data) ** 2))), '.3g')
                        idx_methods += 1
        
            maemse_latex_table = maemse_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
            print(maemse_latex_table)
            maemse_latex_table = maemse_ablation_df.to_latex(index=False, float_format="{:.3f}".format)
            print(maemse_latex_table)

make_latex_table_INDEX_farm()

farm 500
\begin{tabular}{lllllllll}
\toprule
       Method &   0 &    1 &      2 &     3 &      4 &       5 &     6 &     7 \\
\midrule
Proposed (1\%) & 460 &  145 &  0.316 & 0.981 & 0.0438 & 0.00591 & 0.601 & 0.682 \\
Proposed (5\%) & 460 &  150 &  0.326 & 0.946 & 0.0447 & 0.00599 & 0.666 & 0.879 \\
          ORB & 500 &  130 &  0.260 & 0.972 & 0.0655 &  0.0477 &  1.31 &  2.38 \\
         TORB & 500 &  109 &  0.219 & 0.985 & 0.0643 &  0.0449 &  1.26 &  2.11 \\
       SPHORB & 509 & 99.7 &  0.196 & 0.945 & 0.0746 &  0.0646 &  1.25 &  2.25 \\
         SIFT & 500 & 81.5 &  0.163 & 0.965 & 0.0488 & 0.00691 & 0.877 &  1.46 \\
        TSIFT & 494 & 95.2 &  0.193 & 0.971 & 0.0445 & 0.00613 & 0.614 & 0.835 \\
       SPoint & 500 &  100 &  0.200 & 0.989 & 0.0448 & 0.00588 & 0.790 &  1.08 \\
        ALIKE & 500 & 53.9 &  0.108 & 0.985 & 0.0629 &  0.0261 & 0.835 &  1.45 \\
       TALIKE & 500 & 45.9 & 0.0919 & 0.994 & 0.0695 &  0.0231 & 0.758 &  1.26 \\
\bottomrule
\end{tabular}

\begin{tabular}

In [9]:
def make_latex_table_INDEX_farm_old():
    for i, param in enumerate(PARAMS):
            locs = POSES
            print(param)
            maemse_data = {'Points limit': [500, 1000, 10000, 500, 1000, 10000]}
            maemse_compare_df = pd.DataFrame(maemse_data)
            maemse_ablation_df = pd.DataFrame(maemse_data)
            for tgt in ["compare", "ablation"]:
                if tgt == "compare":
                    descriptors = COMPARE_DESCRIPTORS
                    method_tmp_list = []
                    for descriptor in descriptors:
                        for method in METHODS:
                            if descriptor in NOT_T_DESCRIPTORS and method == "t":
                                continue
                            #method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                            method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                            method_tmp_list.append(method_tmp)
                    maemse_data = {'Method': method_tmp_list}
                    maemse_compare_df = pd.DataFrame(maemse_data)
                    maemse_df = maemse_compare_df
                else:
                    descriptors = ABLATION_DESCRIPTORS
                    method_tmp_list = []
                    for descriptor in descriptors:
                        for method in METHODS:
                            if descriptor in NOT_T_DESCRIPTORS and method == "t":
                                continue
                            method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                            method_tmp_list.append(method_tmp)
                    maemse_data = {'Method': method_tmp_list}
                    maemse_ablation_df = pd.DataFrame(maemse_data)
                    maemse_df = maemse_ablation_df


                for method_tmp in method_tmp_list:
                    for j in range(12):
                       maemse_df[j] = [0] * len(method_tmp_list)

                for idx_points, num_points in enumerate([500, 1000, 10000]):
                    base_path = f"results/FP_{num_points}/values"
                    idx_methods = 0
                    for descriptor in descriptors:
                        for method in METHODS:
                            if descriptor in NOT_T_DESCRIPTORS and method == "t":
                                continue
                            all_MN_data = []
                            all_MS_data = []
                            all_error_data = []
                            method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                            for scene in locs:
                                file_base_path = f"{base_path}/{scene}_{method}{descriptor}_5PA_GSM_wRT"
                                MN_path = file_base_path + "/MATCHING_NUM.csv"
                                MN_data = read_csv_data(MN_path)
                                all_MN_data.extend(MN_data)
                                MS_path = file_base_path + "/MATCHING_SCORE.csv"
                                MS_data = read_csv_data(MS_path)
                                all_MS_data.extend(MS_data)
                                error_path = file_base_path + f"/{param}_ERRORS.csv"
                                error_data = read_csv_data(error_path)
                                all_error_data.extend(error_data)
                            
                            maemse_df.loc[idx_methods, idx_points*4] = format(Decimal(str(np.mean(all_MN_data))), '.3g')
                            maemse_df.loc[idx_methods, idx_points*4+1] = format(Decimal(str(np.mean(all_MS_data))), '.3g')
                            maemse_df.loc[idx_methods, idx_points * 4+2] = format(Decimal(str(np.mean(all_error_data))), '.3g')
                            maemse_df.loc[idx_methods, idx_points * 4+3] = format(Decimal(str(np.mean(np.array(all_error_data) ** 2))), '.3g')
                            idx_methods += 1
        
            maemse_latex_table = maemse_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
            print(maemse_latex_table)
            maemse_latex_table = maemse_ablation_df.to_latex(index=False, float_format="{:.3f}".format)
            print(maemse_latex_table)

make_latex_table_INDEX_farm_old()

R
\begin{tabular}{lllllllllllll}
\toprule
       Method &    0 &      1 &      2 &       3 &    4 &      5 &      6 &       7 &       8 &      9 &     10 &      11 \\
\midrule
Proposed (1\%) &  145 &  0.306 & 0.0438 & 0.00591 &  151 &  0.315 & 0.0445 & 0.00600 &     151 &  0.315 & 0.0445 & 0.00600 \\
Proposed (5\%) &  150 &  0.315 & 0.0447 & 0.00599 &  155 &  0.325 & 0.0451 & 0.00602 &     155 &  0.325 & 0.0451 & 0.00602 \\
          ORB &  130 & 0.0240 & 0.0655 &  0.0477 &  263 & 0.0486 & 0.0490 & 0.00663 & 1.02e+3 &  0.179 & 0.0544 & 0.00755 \\
         TORB &  109 & 0.0310 & 0.0643 &  0.0449 &  234 & 0.0661 & 0.0474 & 0.00636 &     761 &  0.196 & 0.0521 & 0.00730 \\
       SPHORB & 99.7 &  0.208 & 0.0746 &  0.0646 &  204 &  0.428 & 0.0483 & 0.00644 &     868 &   1.84 & 0.0510 & 0.00672 \\
         SIFT & 81.5 & 0.0717 & 0.0488 & 0.00691 &  120 &  0.102 & 0.0439 & 0.00629 &     149 &  0.119 & 0.0434 & 0.00629 \\
        TSIFT & 95.2 &  0.136 & 0.0445 & 0.00613 &  131 &  0.179 & 0.043

In [6]:
def make_latex_table_MAEMSE_farm():
    for i, param in enumerate(PARAMS):
        print(param)
        maemse_data = {'Points limit': [500, 1000, 10000, 500, 1000, 10000]}
        maemse_compare_df = pd.DataFrame(maemse_data)
        maemse_ablation_df = pd.DataFrame(maemse_data)
        for tgt in ["compare", "ablation"]:
            if tgt == "compare":
                descriptors = COMPARE_DESCRIPTORS
                
                maemse_df = maemse_compare_df
            else:
                descriptors = ABLATION_DESCRIPTORS
                maemse_df = maemse_ablation_df
            for descriptor in descriptors:
                for method in METHODS:
                    if descriptor in NOT_T_DESCRIPTORS and method == "t":
                        continue
                    method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                    maemse_df[method_tmp] = [0, 0, 0, 0, 0, 0]
    
        for idx_points, num_points in enumerate([500, 1000, 10000]):
            base_path = f"results/FP_{num_points}/values"
            value_dict = {}
            for tgt in ["compare", "ablation"]:
                if tgt == "compare":
                    descriptors = COMPARE_DESCRIPTORS
                    maemse_df = maemse_compare_df
                else:
                    descriptors = ABLATION_DESCRIPTORS
                    maemse_df = maemse_ablation_df
                for descriptor in descriptors:
                    for method in METHODS:
                        if descriptor in NOT_T_DESCRIPTORS and method == "t":
                            continue
                        all_error_data = []
                        for scene in POSES:
                            file_path = f"{base_path}/{scene}_{method}{descriptor}_5PA_GSM_wRT/{param}_ERRORS.csv"
                            error_data = read_csv_data(file_path)
                            all_error_data.extend(error_data)
                        method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                        maemse_df.loc[idx_points, method_tmp] = np.mean(all_error_data)
                        maemse_df.loc[idx_points+3, method_tmp] = np.mean(np.array(all_error_data) ** 2)
    
        maemse_latex_table = maemse_compare_df.to_latex(index=False,  float_format="{:.4f}".format)
        print(maemse_latex_table)
        maemse_latex_table = maemse_ablation_df.to_latex(index=False, float_format="{:.4f}".format)
        print(maemse_latex_table)

make_latex_table_MAEMSE_farm()

R
\begin{tabular}{rrrrrrrrrr}
\toprule
 Points limit &  Proposed (5\%) &    ORB &   TORB &  SPHORB &   SIFT &  TSIFT &  SPoint &  ALIKE &  TALIKE \\
\midrule
          500 &         0.0447 & 0.0655 & 0.0643 &  0.0746 & 0.0488 & 0.0445 &  0.0448 & 0.0571 &  0.0692 \\
         1000 &         0.0451 & 0.0490 & 0.0474 &  0.0483 & 0.0439 & 0.0433 &  0.0445 & 0.0600 &  0.0638 \\
        10000 &         0.0451 & 0.0544 & 0.0521 &  0.0510 & 0.0434 & 0.0432 &  0.0440 & 0.0579 &  0.0743 \\
          500 &         0.0060 & 0.0477 & 0.0449 &  0.0646 & 0.0069 & 0.0061 &  0.0059 & 0.0102 &  0.0230 \\
         1000 &         0.0060 & 0.0066 & 0.0064 &  0.0064 & 0.0063 & 0.0060 &  0.0059 & 0.0146 &  0.0147 \\
        10000 &         0.0060 & 0.0075 & 0.0073 &  0.0067 & 0.0063 & 0.0060 &  0.0059 & 0.0084 &  0.0527 \\
\bottomrule
\end{tabular}

\begin{tabular}{rrrrrrrrr}
\toprule
 Points limit &  Proposed (5\%) &  Proposed (0.1\%) &  Proposed (1\%) &  Proposed (10\%) &    MNN &  Proposed (5\%)+L &  KNN+

C:\Users\taab7\AppData\Local\Temp\ipykernel_13900\2984267133.py:44: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  maemse_latex_table = maemse_compare_df.to_latex(index=False,  float_format="{:.4f}".format)
C:\Users\taab7\AppData\Local\Temp\ipykernel_13900\2984267133.py:46: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  maemse_latex_table = maemse_ablation_df.to_latex(index=False, float_format="{:.4f}".format)
C:\Users\taab7\AppData\Local\Temp\ipykernel_13900\2984267133.py:44

In [7]:
def make_latex_table_time_farm():
    for i, param in enumerate(PARAMS):
        time_data = {'Points limit': ["ALL", 500, 1000, 10000]}
        time_compare_df = pd.DataFrame(time_data)
        time_ablation_df = pd.DataFrame(time_data)
        for tgt in ["compare", "ablation"]:
            if tgt == "compare":
                descriptors = COMPARE_DESCRIPTORS
                time_df = time_compare_df
            else:
                descriptors = ABLATION_DESCRIPTORS
                time_df = time_ablation_df
            for descriptor in descriptors:
                for method in METHODS:
                    if descriptor in NOT_T_DESCRIPTORS and method == "t":
                        continue
                    method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                    time_df[method_tmp] = [0, 0, 0, 0]
    
        for idx_points, num_points in enumerate([500, 1000, 10000]):
            base_path = f"results/FP_{num_points}/values"
            value_dict = {}
            for tgt in ["compare", "ablation"]:
                if tgt == "compare":
                    descriptors = COMPARE_DESCRIPTORS
                    time_df = time_compare_df
                else:
                    descriptors = ABLATION_DESCRIPTORS
                    time_df = time_ablation_df
                for time_category in TIME_CATEGORYS:
                    for descriptor in descriptors:
                        for method in METHODS:
                            if descriptor in NOT_T_DESCRIPTORS and method == "t":
                                continue
                            method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                            all_time_data = []
                            for scene in ALL_LOCS:
                                file_path = f"{base_path}/{scene}_{method}{descriptor}_5PA_GSM_wRT/TIMES_{time_category}.csv"
                                time_data = read_csv_data(file_path)
                                all_time_data.extend(time_data)
                            if time_category == "FP":
                                time_df.loc[0, method_tmp] = np.mean(all_time_data)
                            elif time_category == "MC":
                                time_df.loc[idx_points+1, method_tmp] = np.mean(all_time_data)
    
        time_latex_table = time_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
        print(time_latex_table)
        time_latex_table = time_ablation_df.to_latex(index=False, float_format="{:.3f}".format)
        print(time_latex_table)

make_latex_table_time_farm()

C:\Users\taab7\AppData\Local\Temp\ipykernel_13900\3162801893.py:52: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  time_latex_table = time_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
C:\Users\taab7\AppData\Local\Temp\ipykernel_13900\3162801893.py:54: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  time_latex_table = time_ablation_df.to_latex(index=False, float_format="{:.3f}".format)


\begin{tabular}{lrrrrrrrrr}
\toprule
Points limit &  Proposed (5\%) &   ORB &  TORB &  SPHORB &  SIFT &  TSIFT &  SPoint &  ALIKE &  TALIKE \\
\midrule
         ALL &          0.483 & 0.194 & 0.218 &   0.143 & 0.223 &  0.668 &   0.236 &  0.186 &   0.331 \\
         500 &          0.008 & 0.011 & 0.013 &   0.001 & 0.010 &  0.011 &   0.011 &  0.010 &   0.011 \\
        1000 &          0.013 & 0.010 & 0.013 &   0.002 & 0.009 &  0.010 &   0.011 &  0.010 &   0.011 \\
       10000 &          0.015 & 0.084 & 0.058 &   0.105 & 0.012 &  0.011 &   0.014 &  0.012 &   0.012 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrrrr}
\toprule
Points limit &  Proposed (5\%) &  Proposed (0.1\%) &  Proposed (1\%) &  Proposed (10\%) &   MNN &  Proposed (5\%)+L &  KNN+L &  FLANN \\
\midrule
         ALL &          0.483 &            0.495 &          0.500 &           0.496 & 0.481 &            0.483 &  0.483 &  0.481 \\
         500 &          0.008 &            0.013 &          0.013 &           0.011 & 0

C:\Users\taab7\AppData\Local\Temp\ipykernel_13900\3162801893.py:52: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  time_latex_table = time_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
C:\Users\taab7\AppData\Local\Temp\ipykernel_13900\3162801893.py:54: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  time_latex_table = time_ablation_df.to_latex(index=False, float_format="{:.3f}".format)


\begin{tabular}{lrrrrrrrrr}
\toprule
Points limit &  Proposed (5\%) &   ORB &  TORB &  SPHORB &  SIFT &  TSIFT &  SPoint &  ALIKE &  TALIKE \\
\midrule
         ALL &          0.483 & 0.194 & 0.218 &   0.143 & 0.223 &  0.668 &   0.236 &  0.186 &   0.331 \\
         500 &          0.008 & 0.011 & 0.013 &   0.001 & 0.010 &  0.011 &   0.011 &  0.010 &   0.011 \\
        1000 &          0.013 & 0.010 & 0.013 &   0.002 & 0.009 &  0.010 &   0.011 &  0.010 &   0.011 \\
       10000 &          0.015 & 0.084 & 0.058 &   0.105 & 0.012 &  0.011 &   0.014 &  0.012 &   0.012 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrrrr}
\toprule
Points limit &  Proposed (5\%) &  Proposed (0.1\%) &  Proposed (1\%) &  Proposed (10\%) &   MNN &  Proposed (5\%)+L &  KNN+L &  FLANN \\
\midrule
         ALL &          0.483 &            0.495 &          0.500 &           0.496 & 0.481 &            0.483 &  0.483 &  0.481 \\
         500 &          0.008 &            0.013 &          0.013 &           0.011 & 0

C:\Users\taab7\AppData\Local\Temp\ipykernel_13900\3162801893.py:52: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  time_latex_table = time_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
C:\Users\taab7\AppData\Local\Temp\ipykernel_13900\3162801893.py:54: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  time_latex_table = time_ablation_df.to_latex(index=False, float_format="{:.3f}".format)


\begin{tabular}{lrrrrrrrrr}
\toprule
Points limit &  Proposed (5\%) &   ORB &  TORB &  SPHORB &  SIFT &  TSIFT &  SPoint &  ALIKE &  TALIKE \\
\midrule
         ALL &          0.483 & 0.194 & 0.218 &   0.143 & 0.223 &  0.668 &   0.236 &  0.186 &   0.331 \\
         500 &          0.008 & 0.011 & 0.013 &   0.001 & 0.010 &  0.011 &   0.011 &  0.010 &   0.011 \\
        1000 &          0.013 & 0.010 & 0.013 &   0.002 & 0.009 &  0.010 &   0.011 &  0.010 &   0.011 \\
       10000 &          0.015 & 0.084 & 0.058 &   0.105 & 0.012 &  0.011 &   0.014 &  0.012 &   0.012 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrrrr}
\toprule
Points limit &  Proposed (5\%) &  Proposed (0.1\%) &  Proposed (1\%) &  Proposed (10\%) &   MNN &  Proposed (5\%)+L &  KNN+L &  FLANN \\
\midrule
         ALL &          0.483 &            0.495 &          0.500 &           0.496 & 0.481 &            0.483 &  0.483 &  0.481 \\
         500 &          0.008 &            0.013 &          0.013 &           0.011 & 0

C:\Users\taab7\AppData\Local\Temp\ipykernel_13900\3162801893.py:52: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  time_latex_table = time_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
C:\Users\taab7\AppData\Local\Temp\ipykernel_13900\3162801893.py:54: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  time_latex_table = time_ablation_df.to_latex(index=False, float_format="{:.3f}".format)
